In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline

In [233]:
dataset = pd.read_csv('HR100.txt', delimiter = '\t', quoting = 3)

In [234]:
dataset.drop('Count', axis=1).head(5)

,Name,KPIs,InPreferredDept,CurrDept
0,Tarial,Performed VSS on CISCO catalyst switches and r...,0,Network
1,Anastasia,Conducts organizational assessments to priorit...,1,Security
2,Regis,Designed business analtytics and reporting tha...,0,Data
3,Brent,Developed and implemented complex Internet and...,0,WebDev
4,Saini,Provided job placement accomodations for emply...,0,HR


In [235]:
df = dataset.drop('CurrDept', axis=1)
df = dataset.drop('Count', axis=1)

In [236]:
import re
import nltk 
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [237]:
corpus = []
stored_corpus = []

for i in range(0, 49):
    individual_corpus = []
    details = re.sub('[^a-zA-Z0-9]', ' ', dataset['KPIs'][i])
    details = details.lower()
    details = details.split()
    #new_stopwords = ['class', '10th', '12th']
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    #all_stopwords.extend(new_stopwords)
    all_stopwords.remove('not')
    details = [ps.stem(word) for word in details if not word in set(all_stopwords)]
    details = ' '.join(details)
    individual_corpus.append(details)
    stored_corpus.append(individual_corpus)
    #print(individual_corpus)
    corpus.append(details)
    corpus.append(details)

In [238]:
emp_class = df[(df.InPreferredDept ==1) | (df.InPreferredDept==0)]

In [239]:
emp_class.head(5)

,Name,KPIs,InPreferredDept,CurrDept
0,Tarial,Performed VSS on CISCO catalyst switches and r...,0,Network
1,Anastasia,Conducts organizational assessments to priorit...,1,Security
2,Regis,Designed business analtytics and reporting tha...,0,Data
3,Brent,Developed and implemented complex Internet and...,0,WebDev
4,Saini,Provided job placement accomodations for emply...,0,HR


In [240]:
def text_process(mess):
    ps = PorterStemmer()
    # Checking for punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    # Removing stopwords and stemming
    return [ps.stem(word.lower()) for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [241]:
X = emp_class['KPIs']
y = emp_class['InPreferredDept']

In [242]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [243]:
Z = []
for i in range(0, df.shape[0]):
    s = ''.join(text_process(df['KPIs'][i]))
    Z.append(s)
X = cv.fit_transform(Z)

In [244]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30,random_state=52)

In [245]:
import tensorflow as tf

In [246]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [247]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

In [248]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', mode='max', verbose=1, patience=15)

In [249]:
X_train = X_train.toarray()

In [250]:
ann.fit(X_train, y_train, batch_size = 4, epochs = 20, verbose = 1, callbacks=[early_stop])

Epoch 1/20
18/18 [==============================] - 1s 3ms/step - loss: 0.6598 - binary_accuracy: 0.7571
Epoch 2/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6407 - binary_accuracy: 0.8000
Epoch 3/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6227 - binary_accuracy: 0.7857
Epoch 4/20
18/18 [==============================] - 0s 2ms/step - loss: 0.6055 - binary_accuracy: 0.7857
Epoch 5/20
18/18 [==============================] - 0s 2ms/step - loss: 0.5876 - binary_accuracy: 0.7714
Epoch 6/20
18/18 [==============================] - 0s 2ms/step - loss: 0.5716 - binary_accuracy: 0.7714
Epoch 7/20
18/18 [==============================] - 0s 2ms/step - loss: 0.5553 - binary_accuracy: 0.7714
Epoch 8/20
18/18 [==============================] - 0s 2ms/step - loss: 0.5395 - binary_accuracy: 0.7714
Epoch 9/20
18/18 [==============================] - 0s 2ms/step - loss: 0.5243 - binary_accuracy: 0.7714
Epoch 10/20
18/18 [==============================] - 0s

In [251]:
print(type(X_test))


<class 'scipy.sparse.csr.csr_matrix'>


In [252]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

X_test = convert_sparse_matrix_to_sparse_tensor(X_test)

In [257]:
y_pred = ann.predict(tf.sparse.reorder(X_test))
y_pred
y_pred = (y_pred > 0.30)


1/1 [==============================] - 0s 17ms/step


In [258]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

[[20  4]
 [ 3  3]]
              precision    recall  f1-score   support

           0       0.87      0.83      0.85        24
           1       0.43      0.50      0.46         6

    accuracy                           0.77        30
   macro avg       0.65      0.67      0.66        30
weighted avg       0.78      0.77      0.77        30

